In [27]:
import pandas as pd

In [28]:
# Read the Excel file
file_path = "rawMeterData-123W93Street.xlsx"  # Ensure the file path is correct
xls = pd.ExcelFile(file_path)

# Read all sheets (2021-2024)
data_frames = []
for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)  # Parse each sheet
    df['Year'] = sheet_name  # Add a Year column to indicate the sheet's year
    data_frames.append(df)

# Merge data from all years into a single DataFrame
df_all = pd.concat(data_frames, ignore_index=True)

# Check the data format
df_all.head()  # Display the first few rows of the dataset

# Export to Excel file
df_all.to_excel("combined_data.xlsx", index=False)

In [31]:
# Read Excel file
df = pd.read_excel("combined_data.xlsx")

# Convert the 'dateTimeStmp' column to datetime format
df['dateTimeStmp'] = pd.to_datetime(df['dateTimeStmp'])

# Extract the date portion (excluding the specific time).
df['date'] = df['dateTimeStmp'].dt.date

# Group by date to filter out data where all records in a day are not 0
NONZERO_SUBSET = df.groupby('date').filter(
    lambda x: (x['raw_MM_Wh'] != 0).all() and (x['raw_Tenant_Wh'] != 0).all()
)

# 5. Keep the required columns
NONZERO_SUBSET = NONZERO_SUBSET[['dateTimeStmp', 'raw_MM_Wh', 'raw_Tenant_Wh']]

# Define a function to convert a date to a string with ordinal suffix
def ordinal(n):
    if 11 <= n % 100 <= 13:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return str(n) + suffix

# Define the function to format the date
def format_date(dt):
    return dt.strftime('%b ') + ordinal(dt.day) + dt.strftime(' %Y')

# Add new column 'Exact_Date' to display the formatted date
NONZERO_SUBSET['Exact_Date'] = NONZERO_SUBSET['dateTimeStmp'].apply(format_date)

# Count the number of 0s in each column
zero_counts = (NONZERO_SUBSET[['raw_MM_Wh', 'raw_Tenant_Wh']] == 0).sum()
print("The number of zeros in each column：")
print(zero_counts)

# Export to Excel file
NONZERO_SUBSET.to_excel("NONZERO_SUBSET.xlsx", index=False)

# Confirm the output result.
print("Export successful, total number of records：", NONZERO_SUBSET.shape[0])


The number of zeros in each column：
raw_MM_Wh        0
raw_Tenant_Wh    0
dtype: int64
Export successful, total number of records： 117604
